#### Loading Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('darts.csv')

In [3]:
data.head()

,xCoord,yCoord,competitor
0,0.196451,-0.520341,Steve
1,0.476027,-0.306763,Susan
2,0.003175,-0.980736,Michael
3,0.294078,0.267566,Kate
4,-0.051120,0.598946,Steve


In [4]:
data.dtypes

xCoord        float64
yCoord        float64
competitor     object
dtype: object

In [5]:
# Describe the data
print('Dataset stats: \n', data.describe())

Dataset stats: 
            xCoord      yCoord
count  800.000000  800.000000
mean     0.005943    0.022868
std      0.475971    0.514482
min     -0.991140   -0.993547
25%     -0.295314   -0.423079
50%      0.000619    0.034483
75%      0.273953    0.452204
max      0.984816    0.994344


In [6]:
# Count the number of observations of each class
print('Observations per class: \n', data['competitor'].value_counts())

Observations per class: 
 Kate       200
Steve      200
Susan      200
Michael    200
Name: competitor, dtype: int64


#### Data preparation

In [7]:
from keras.utils import to_categorical

# Transform into a categorical variable
data.competitor = pd.Categorical(data.competitor)

# Assign a number to each category (label encoding)
data.competitor = data.competitor.cat.codes 

# Print the label encoded competitors
print('Label encoded competitors: \n',data.competitor.head())

Using TensorFlow backend.


Label encoded competitors: 
 0    2
1    3
2    1
3    0
4    2
Name: competitor, dtype: int8


In [8]:
# Use to_categorical on your labels
coordinates = data.drop(['competitor'], axis=1)
competitors = to_categorical(data.competitor)

# Now print the to_categorical() result
print('One-hot encoded competitors: \n',competitors)

One-hot encoded competitors: 
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [9]:
from sklearn.model_selection import train_test_split
y = to_categorical(data.competitor)
X = data.loc[:, data.columns != 'competitor']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

#### Model configuration

In [10]:
# Import the Sequential model and Dense layer
from keras.models import Sequential
from keras.layers import Dense

# Create a sequential model
model = Sequential()

# Add 3 dense layers of 128, 64 and 32 neurons each
model.add(Dense(128, input_shape=(2,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Add a dense layer with as many neurons as competitors
model.add(Dense(4, activation='softmax'))

# Display a summary of your model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 132       
Total params: 10,852
Trainable params: 10,852
Non-trainable params: 0
_________________________________________________________________


#### Model compilation

In [11]:
# Compile your model using categorical_crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Model tranining

In [12]:
# Train your model for 20 epochs
model.fit(X_train, y_train, epochs=20)


Epoch 1/20
640/640 [==============================] - 0s 574us/step - loss: 1.3815 - accuracy: 0.2188
Epoch 2/20
640/640 [==============================] - 0s 114us/step - loss: 1.3494 - accuracy: 0.2719
Epoch 3/20
640/640 [==============================] - 0s 79us/step - loss: 1.3087 - accuracy: 0.3094
Epoch 4/20
640/640 [==============================] - 0s 80us/step - loss: 1.2499 - accuracy: 0.3844
Epoch 5/20
640/640 [==============================] - 0s 77us/step - loss: 1.1709 - accuracy: 0.4906
Epoch 6/20
640/640 [==============================] - 0s 94us/step - loss: 1.0744 - accuracy: 0.5547
Epoch 7/20
640/640 [==============================] - 0s 78us/step - loss: 0.9684 - accuracy: 0.5750
Epoch 8/20
640/640 [==============================] - 0s 91us/step - loss: 0.8896 - accuracy: 0.6313
Epoch 9/20
640/640 [==============================] - 0s 92us/step - loss: 0.8436 - accuracy: 0.6438
Epoch 10/20
640/640 [==============================] - 0s 75us/step - loss: 0.8084 - acc

#### Model evaluation

In [13]:
# Evaluate your model accuracy on the test set
accuracy = model.evaluate(X_test, y_test)

# Print accuracy
print('Accuracy:',accuracy)

160/160 [==============================] - 0s 413us/step
Accuracy: [0.9348755359649659, 0.675000011920929]


#### Predicting

In [14]:
preds = model.predict(X_test)

In [15]:
# Print preds vs true values
print("{:45} | {}".format('Raw Model Predictions','True labels'))
for i,pred in enumerate(preds):
  print("{} | {}".format(pred,y_test[i]))

Raw Model Predictions                         | True labels
[3.2137215e-01 9.0283247e-05 6.6736096e-01 1.1176597e-02] | [0. 0. 0. 1.]
[5.0363461e-03 8.1229937e-01 5.0808617e-06 1.8265916e-01] | [0. 1. 0. 0.]
[0.08888436 0.4815438  0.00196232 0.42760947] | [0. 0. 0. 1.]
[1.5489795e-03 8.8409799e-01 1.5690659e-05 1.1433737e-01] | [0. 1. 0. 0.]
[2.1041862e-03 8.8259244e-01 3.0141506e-05 1.1527318e-01] | [0. 1. 0. 0.]
[0.41299507 0.02312833 0.10090866 0.46296796] | [1. 0. 0. 0.]
[4.1589108e-03 7.9665989e-01 4.0191429e-05 1.9914101e-01] | [0. 1. 0. 0.]
[5.8010435e-03 7.2486913e-01 6.8173773e-05 2.6926169e-01] | [0. 0. 0. 1.]
[0.58314717 0.00072693 0.36049432 0.05563156] | [0. 1. 0. 0.]
[0.66821635 0.00837075 0.07617462 0.24723826] | [1. 0. 0. 0.]
[0.05884616 0.34143722 0.00187742 0.5978392 ] | [0. 0. 0. 1.]
[3.6826220e-01 1.1147702e-04 6.1758816e-01 1.4038199e-02] | [0. 0. 1. 0.]
[0.1887523  0.19297546 0.03586762 0.5824046 ] | [0. 0. 0. 1.]
[0.4937189  0.0011461  0.44647652 0.05865853] | [1

In [16]:
# Extract the indexes of the highest probable predictions
preds = [np.argmax(pred) for pred in preds]

# Print preds vs true values
print("{:10} | {}".format('Rounded Model Predictions','True labels'))
for i,pred in enumerate(preds):
  print("{:25} | {}".format(pred,y_test[i]))

Rounded Model Predictions | True labels
                        2 | [0. 0. 0. 1.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 0. 0. 1.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        3 | [1. 0. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 0. 0. 1.]
                        0 | [0. 1. 0. 0.]
                        0 | [1. 0. 0. 0.]
                        3 | [0. 0. 0. 1.]
                        2 | [0. 0. 1. 0.]
                        3 | [0. 0. 0. 1.]
                        0 | [1. 0. 0. 0.]
                        0 | [1. 0. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        1 | [0. 1. 0. 0.]
                        1 | [1. 0. 0. 0.]
                        2 | [1. 0. 0. 0.]
                        0 | [1. 0. 0. 0.]
                        1 | [0. 1. 0